### REG No: 8569202378

### NAME: Ashvant Ram Selvam

Concat the text files to one.

In [27]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving weights.best (1).hdf5 to weights.best (1) (1).hdf5
User uploaded file "weights.best (1).hdf5" with length 825328 bytes


i. Concatenate your text files to create a corpus of Russell’s writings.

The books in the data set are read and concatenated into the full data file

In [1]:
filenames = ['book1.txt','book2.txt','book3.txt','book4.txt']
with open('fullData.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [2]:
inputFile = 'fullData.txt'
fread = open(inputFile)
dataArray = []
for line in fread:
    line = line.strip()
    for code in line.encode('ascii','ignore'):
        dataArray.append(code)

ii. Use a character-level representation for this model by using extended ASCII
that has N = 256 characters. Each character will be encoded into a an integer
using its ASCII code. Rescale the integers to the range [0, 1], because LSTM uses a sigmoid activation function. LSTM will receive the rescaled integers
as its input

ASCII encoded values

In [3]:
dataArray

[80,
 114,
 111,
 100,
 117,
 99,
 101,
 100,
 32,
 98,
 121,
 32,
 71,
 111,
 114,
 100,
 111,
 110,
 32,
 75,
 101,
 101,
 110,
 101,
 114,
 84,
 72,
 69,
 32,
 80,
 82,
 79,
 66,
 76,
 69,
 77,
 83,
 32,
 79,
 70,
 32,
 80,
 72,
 73,
 76,
 79,
 83,
 79,
 80,
 72,
 89,
 66,
 121,
 32,
 66,
 101,
 114,
 116,
 114,
 97,
 110,
 100,
 32,
 82,
 117,
 115,
 115,
 101,
 108,
 108,
 80,
 82,
 69,
 70,
 65,
 67,
 69,
 73,
 110,
 32,
 116,
 104,
 101,
 32,
 102,
 111,
 108,
 108,
 111,
 119,
 105,
 110,
 103,
 32,
 112,
 97,
 103,
 101,
 115,
 32,
 73,
 32,
 104,
 97,
 118,
 101,
 32,
 99,
 111,
 110,
 102,
 105,
 110,
 101,
 100,
 32,
 109,
 121,
 115,
 101,
 108,
 102,
 32,
 105,
 110,
 32,
 116,
 104,
 101,
 32,
 109,
 97,
 105,
 110,
 32,
 116,
 111,
 32,
 116,
 104,
 111,
 115,
 101,
 112,
 114,
 111,
 98,
 108,
 101,
 109,
 115,
 32,
 111,
 102,
 32,
 112,
 104,
 105,
 108,
 111,
 115,
 111,
 112,
 104,
 121,
 32,
 105,
 110,
 32,
 114,
 101,
 103,
 97,
 114,
 100,
 32,
 116,
 111,
 32,

Features Scaled to [0,1]

In [4]:
from sklearn.preprocessing import MinMaxScaler


In [5]:
import numpy as np
norm = MinMaxScaler(feature_range = (0, 1))
dataArrayNormalized = norm.fit_transform(np.array(dataArray).reshape(-1, 1))

/Users/ashvant/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
len(dataArrayNormalized)

241183

iii. Choose a window size, e.g., W = 100.

iv. Inputs to the network will be the first W −1 = 99 characters of each sequence,
and the output of the network will be the L
th character of the sequence.
Basically, we are training the network to predict the each character using the
99 characters that precede it. Slide the window in strides of S = 1 on the
text. For example, if W = 5 and S = 1 and we want to train the network
with the sequence ABRACADABRA, The first input to the network will be
ABRA and the corresponding output will be C. The second input will be
BRAC and the second output will be A, etc.

trainData has the 99 character sequence and trainLabels has the 100th character

In [7]:
trainData = []
trainLabels = []
for i in range(99, len(dataArrayNormalized)):
    trainData.append(dataArrayNormalized[i-99:i])
    trainLabels.append(dataArray[i])

In [8]:
trainLabels

[32,
 73,
 32,
 104,
 97,
 118,
 101,
 32,
 99,
 111,
 110,
 102,
 105,
 110,
 101,
 100,
 32,
 109,
 121,
 115,
 101,
 108,
 102,
 32,
 105,
 110,
 32,
 116,
 104,
 101,
 32,
 109,
 97,
 105,
 110,
 32,
 116,
 111,
 32,
 116,
 104,
 111,
 115,
 101,
 112,
 114,
 111,
 98,
 108,
 101,
 109,
 115,
 32,
 111,
 102,
 32,
 112,
 104,
 105,
 108,
 111,
 115,
 111,
 112,
 104,
 121,
 32,
 105,
 110,
 32,
 114,
 101,
 103,
 97,
 114,
 100,
 32,
 116,
 111,
 32,
 119,
 104,
 105,
 99,
 104,
 32,
 73,
 32,
 116,
 104,
 111,
 117,
 103,
 104,
 116,
 32,
 105,
 116,
 32,
 112,
 111,
 115,
 115,
 105,
 98,
 108,
 101,
 32,
 116,
 111,
 32,
 115,
 97,
 121,
 115,
 111,
 109,
 101,
 116,
 104,
 105,
 110,
 103,
 32,
 112,
 111,
 115,
 105,
 116,
 105,
 118,
 101,
 32,
 97,
 110,
 100,
 32,
 99,
 111,
 110,
 115,
 116,
 114,
 117,
 99,
 116,
 105,
 118,
 101,
 44,
 32,
 115,
 105,
 110,
 99,
 101,
 32,
 109,
 101,
 114,
 101,
 108,
 121,
 32,
 110,
 101,
 103,
 97,
 116,
 105,
 118,
 101,
 32,
 99,
 

In [9]:
len(trainLabels)

241084

v. Note that the output has to be encoded using a one-hot encoding scheme with
N = 256 (or less) elements. This means that the network reads integers, but
outputs a vector of N = 256 (or less) elements.

In [10]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(n_values = 256)
trainLabelEncoded = ohe.fit_transform(np.array(trainLabels).reshape(-1, 1))


One hot encoding of Labels in trainLabelEncoded

In [11]:
trainLabelEncoded.shape

(241084, 256)

vi. Use a single hidden layer for the LSTM with N = 256 (or less) memory units.
vii. Use a Softmax output layer to yield a probability prediction for each of the
characters between 0 and 1. This is actually a character classification problem
with N classes. Choose log loss (cross entropy) as the objective function for
the network (research what it menas).
viii. We do not use a test dataset. We are using the whole training dataset to
learn the probability of each character in a sequence. We are not seeking for
a very accurate model of. Instead we are interested in a generalization of the
dataset that can mimic the gist of the text.

x. Use model checkpointing to keep the network weights to determine each time
an improvement in loss is observed at the end of the epoch. Find the best set
of weights in terms of loss.

In [14]:
trainData = np.array(trainData)

In [15]:
trainData = np.reshape(trainData, (trainData.shape[0], trainData.shape[1], 1))

In [16]:
from keras.utils import np_utils
newLabels = np_utils.to_categorical(trainLabels,num_classes=256)

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [18]:
from keras.layers import Flatten

LSTM Construction with 256 hidden units and softmax activation and categorical_crossentropy as the loss function.

In [19]:
model = Sequential()

model.add(LSTM(units = 100, return_sequences = False, input_shape = (trainData.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 256))
model.add(Dropout(0.2))
model.add(Dense(units = 256, activation='softmax'))
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy')

In [36]:
model.load_weights("weights.bestlstm.hdf5")

ix. Choose a reasonable number of epochs for training (e.g., 30, although the
network will need more epochs to yield a better model).

The model was repeatedly trained for 200 epochs and the weights were stored and loaded for subsequent evaluation

model.fit(trainData,newLabels, validation_split=0.33, epochs=40, batch_size=4000, callbacks=callbacks_list, verbose=1)

x. Use model checkpointing to keep the network weights to determine each time
an improvement in loss is observed at the end of the epoch. Find the best set
of weights in terms of loss

In [12]:
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [13]:
filepath="weights.bestnew.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Weights stored through model checkpointing are loaded for evaluation.

In [0]:
model.load_weights("weights.best.hdf5")

In [0]:
files.download("weights.best.hdf5")

xi. Use the network with the best weights to generate 1000 characters, using the
following text as initialization of the network:
There are those who take mental phenomena naively, just as they
would physical phenomena. This school of psychologists tends not to
emphasize the object.

In [37]:
outputString = "There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object"

In [38]:
testData = []
outputString = outputString.strip()
i = 0
for code in outputString.encode('ascii','ignore'):
        if(i<99):
          testData.append(code)
          i=i+1

In [39]:
testSample = []
testSample.append(testData[:99])

In [40]:
testDataTransform = norm.transform(testSample)

In [41]:
testSampleShape = np.reshape(testDataTransform, (testDataTransform.shape[0], testDataTransform.shape[1], 1))
output = model.predict(testSampleShape)

In [44]:
testData.append(np.where(output[0]==max(output[0]))[0][0])
for i in range(1000):
  testSample = []
  testSample.append(testData[1+i:100+i])
  testDataTransform = norm.transform(testSample)
  testSampleShape = np.reshape(testDataTransform, (testDataTransform.shape[0], testDataTransform.shape[1], 1))
  output = model.predict(testSampleShape)
  testData.append(np.where(output[0]==max(output[0]))[0][0])

In [45]:
predicted = ""
for i in range(len(testData)):
  predicted = predicted + chr(testData[i])
print(predicted)

There are those who take mental phenomena naively, just as they would physical phenomena. This scho th the tael th the tael th the tael th the tael th the tael th the tael that th the taelg of the tannth the tael th the tael th the tael th the tael th the tael th the tael that th the taelg of the tanee  oh ttael th the tael th the tael th the taelgthathe taelgthathe taelgthat thethe taelg of the tane  oah thi e thethe thel th the tael th the tael        th      n   th      n th t   ah t  te the sant tah  the taseat   th t thethe tael th the tael taataI11hewaaI11heaahicaaTTPhehechetteutheihete  toenethentthinghe en taahethaert   that thethe tael that     e       e                e iinience    tahi     n    n    tn the     n   oaahec ie     the  the  taaaaasaa1111eee17111111111111heee eed  iaTthec taaaihaaaaheaaihichidatTTPheaahic   to taTTThe sahictahang         e    II IIRR)ITTRE  rthittin      the                         tr  tstaaheche     the         taaaiI111ees17e do edddheteeihihe

The obove are the 1000 subsequent characters printed through LSTM.